<a href="https://colab.research.google.com/github/fdeiab/loan-default-prediction/blob/main/Loan_Default_Predicition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Background

Loan approval is a critical decision for any lending institution. One of two scenarios may happen, the bank loses out on potential income by rejecting a loan to an individual or a company. Or, they lend a loan to a party exhibiting risky behaviour where heavy losses. This uncertainty is why it is a good idea that banks utilize good loan default prediction models as they stand much to gain. 

**The major goal of this notebook is to assess if a loan should be approved and predict whether a loan will default.**

# The Data

The [dataset](https://www.kaggle.com/datasets/larsen0966/sba-loans-case-data-set) used in this project, "SBA Case", is a subset of the "National SBA" data, which contains historical data from 1987 through 2014. The subset dataset focuses solely on the state of California through the years of 1989 to 2012. The dataset used contains **2,102** records under **35** attributes:

| **Variable Name**   | **Description of Variable**                            |
|:-------------------:|:------------------------------------------------------:|
|  LoanNr\_ChkDgt     |  Identifier – Primary key                              |
|  Name               |  Borrower name                                         |
|  City               |  Borrower city                                         |
|  State              |  Borrower state                                        |
|  Zip                |  Borrower zip code                                     |
|  Bank               |  Bank name                                             |
|  BankState          |  Bank state                                            |
|  NAICS              |  North American industry classification system code    |
|  ApprovalDate       |  Date SBA commitment issued                            |
|  ApprovalFY         |  Fiscal year of commitment                             |
|  Term               |  Loan term in months                                   |
|  NoEmp              |  Number of business employees                          |
|  NewExist           |  1 = Existing business, 2 = New business               |
|  CreateJob          |  Number of jobs created                                |
|  RetainedJob        |  Number of jobs retained                               |
|  FranchiseCode      |  Franchise code, \(00000 or 00001\) = No franchise     |
|  UrbanRural         |  1 = Urban, 2 = rural, 0 = undefined                   |
|  RevLineCr          |  Revolving line of credit: Y = Yes, N = No             |
|  LowDoc             |  LowDoc Loan Program: Y = Yes, N = No                  |
|  ChgOffDate         |  The date when a loan is declared to be in default     |
|  DisbursementDate   |  Disbursement date                                     |
|  DisbursementGross  |  Amount disbursed                                      |
|  BalanceGross       |  Gross amount outstanding                              |
|  MIS\_Status        |  Loan status charged off = CHGOFF, Paid in full = PIF  |
|  ChgOffPrinGr       |  Charged\-off amount                                   |
|  GrAppv             |  Gross amount of loan approved by bank                 |
|  SBA\_Appv          |  SBA's guaranteed amount of approved loan              |
|  New         |  =1 if NewExist=2 \(New Business\), =0 if NewExist=1 \(Existing Business\)                                                                  |
|  Portion     |  Proportion of gross amount guaranteed by SBA                                                                                               |
|  RealEstate  |  =1 if loan is backed by real estate, =0 otherwise                                                                                          |
|  Recession   |  =1 if loan is active during Great Recession, =0 otherwise                                                                                  |
|  Selected    |  =1 if the data are selected as training data to build model for assignment, =0 if the data are selected as testing data to validate model  |
|  Default     |  =1 if MIS\_Status=CHGOFF, =0 if MIS\_Status=PIF                                                                                            |
|  daysterm    |  Extra variable generated when creating “Recession” in Section 4\.1\.6                                                                      |
|  xx          |  Extra variable generated when creating “Recession” in Section 4\.1\.6                                                                      |


Logistic Regression will be used for this project to predict odds ratios and probabilities of a loan defaulting or not.

Target value for this project: **MIS\_Status** / **Default**

In [3]:
#@title #Importing Libraries & Loading The Dataset 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_csv('/content/drive/MyDrive/SBAcase.11.13.17.csv')



# Step 1: Exploratory Data Analysis

First, we'll examine the number of null values present, the column data types, and the summary statistics to tell us more about the dataset.


In [6]:
df.info()
df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2102 entries, 0 to 2101
Data columns (total 35 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Selected           2102 non-null   int64  
 1   LoanNr_ChkDgt      2102 non-null   int64  
 2   Name               2102 non-null   object 
 3   City               2102 non-null   object 
 4   State              2102 non-null   object 
 5   Zip                2102 non-null   int64  
 6   Bank               2099 non-null   object 
 7   BankState          2099 non-null   object 
 8   NAICS              2102 non-null   int64  
 9   ApprovalDate       2102 non-null   int64  
 10  ApprovalFY         2102 non-null   int64  
 11  Term               2102 non-null   int64  
 12  NoEmp              2102 non-null   int64  
 13  NewExist           2101 non-null   float64
 14  CreateJob          2102 non-null   int64  
 15  RetainedJob        2102 non-null   int64  
 16  FranchiseCode      2102 

,Selected,LoanNr_ChkDgt,Zip,NAICS,ApprovalDate,ApprovalFY,Term,NoEmp,NewExist,CreateJob,...,ChgOffPrinGr,GrAppv,SBA_Appv,New,RealEstate,Portion,Recession,daysterm,xx,Default
count,2102.000000,2.102000e+03,2102.000000,2102.000000,2102.000000,2102.000000,2102.000000,2102.000000,2101.000000,2102.000000,...,2.102000e+03,2.102000e+03,2.102000e+03,2102.000000,2102.000000,2102.000000,2102.000000,2102.000000,2099.000000,2102.000000
mean,0.500000,4.469172e+09,92698.612274,531630.902950,16179.587060,2004.035680,126.980495,10.150809,1.153736,2.549952,...,2.002908e+04,2.330641e+05,1.891759e+05,0.154139,0.274500,0.671055,0.068506,3809.414843,20076.896141,0.326356
std,0.500119,2.530069e+09,1878.208435,521.836986,1454.931276,4.006321,93.798944,34.402420,0.362099,8.010175,...,7.543229e+04,3.436310e+05,2.989268e+05,0.361168,0.446368,0.186519,0.252673,2813.968318,2854.510377,0.468991
min,0.000000,1.004285e+09,65757.000000,531110.000000,10554.000000,1989.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000e+00,4.500000e+03,2.250000e+03,0.000000,0.000000,0.296770,0.000000,0.000000,11524.000000,0.000000
25%,0.000000,2.392978e+09,91402.000000,531210.000000,15695.750000,2003.000000,60.000000,2.000000,1.000000,0.000000,...,0.000000e+00,3.000000e+04,1.500000e+04,0.000000,0.000000,0.500000,0.000000,1800.000000,18316.500000,0.000000
50%,0.500000,3.621730e+09,92559.500000,531312.000000,16556.000000,2005.000000,84.000000,3.000000,1.000000,0.000000,...,0.000000e+00,6.100000e+04,4.168000e+04,0.000000,0.000000,0.500000,0.000000,2520.000000,19270.000000,0.000000
75%,1.000000,6.551607e+09,94127.750000,532230.000000,17149.750000,2007.000000,240.000000,8.000000,1.000000,2.000000,...,1.507350e+04,3.000000e+05,2.397562e+05,0.000000,1.000000,0.850000,0.000000,7200.000000,22335.000000,1.000000
max,1.000000,9.958873e+09,96161.000000,533110.000000,18911.000000,2012.000000,306.000000,650.000000,2.000000,130.000000,...,1.509550e+06,2.350000e+06,2.115000e+06,1.000000,1.000000,1.000000,1.000000,9180.000000,27598.000000,1.000000
